# Utility Functions and Configs

Run below cell befor start capturing packets.

In [1]:
from scapy.all import *
import pandas as pd # Pandas - Create and Manipulate DataFrames
import binascii # Binary to Ascii 

def pkt_callback(pkt):
    pkt.show() # debug statement

# Collect field names from IP/TCP/UDP (These will be columns in DF)
ether_fields = [field.name for field in Ether().fields_desc]
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]
icmp_fields = [field.name for field in ICMP().fields_desc]


dataframe_fields = ether_fields + ip_fields + ['time'] + tcp_fields + icmp_fields + ['payload','payload_raw','payload_hex']

def pretty(pcap):
    # Create blank DataFrame
    df = pd.DataFrame(columns=dataframe_fields)
    
    for packet in pcap[IP]:
        # Field array for each row of DataFrame
        field_values = []
        for field in ether_fields:
            if field == 'type':
                if(packet[Ether].fields[field]== 2048):
                    field_values.append('IP')
                else :
                    field_values.append(packet[Ether].fields[field])
            else:
                field_values.append(packet[Ether].fields[field])
        # Add all IP fields to dataframe
        for field in ip_fields:
            if field == 'options':
                # Retrieving number of options defined in IP Header
                field_values.append(len(packet[IP].fields[field]))
            elif field == 'proto':
                if(packet[IP].fields[field]== 17):
                    field_values.append('udp')
                elif(packet[IP].fields[field]== 6):
                    field_values.append('tcp')
                elif(packet[IP].fields[field]==1):
                    field_values.append('icmp')
                else :
                    field_values.append(packet[IP].fields[field])
            else:
                field_values.append(packet[IP].fields[field])
    
        field_values.append(packet.time)
    
        layer_type = type(packet[IP].payload)
        for field in tcp_fields:
            try:
                if field == 'options':
#                    field_values.append(len(packet[layer_type].fields[field]))
                    field_values.append(packet[layer_type].fields[field][0])
                elif field == 'seq':
                    seq_num = str(int(packet[layer_type].fields[field])%10000)
                    field_values.append(seq_num)
                elif field == 'ack':
                    ack_num = str(int(packet[layer_type].fields[field])%10000)
                    field_values.append(ack_num)
                else:
                    field_values.append(packet[layer_type].fields[field])
            except Exception, e:
                field_values.append(None)
            #except:
            #    field_values.append(None)

        
        for field in icmp_fields:
            try:
                if field == 'options':
                    field_values.append(packet[layer_type].fields[field][0])
                elif field == 'code':
                    if packet[layer_type].fields[field] == 3 :
                        field_values.append('port-unreachable')
                    else :
                        field_values.append(packet[layer_type].fields[field])
                elif field == 'type':
                    if packet[layer_type].fields[field] == 3 :
                        field_values.append('dest-unreach')
                    elif packet[layer_type].fields[field] == 0 :
                        field_values.append('Echo Reply')
                    elif packet[layer_type].fields[field] == 8 :
                        field_values.append('Echo Request')
                    else :
                        field_values.append(packet[layer_type].fields[field])
                else:
                    field_values.append(packet[layer_type].fields[field])
            except:
                field_values.append(None)
    
        # Append payload
        field_values.append(len(packet[layer_type].payload))
        field_values.append(packet[layer_type].payload.original)
        field_values.append(binascii.hexlify(packet[layer_type].payload.original))
        # Add row to DF
        df_append = pd.DataFrame([field_values], columns=dataframe_fields)
        df = pd.concat([df, df_append], axis=0)
    # Reset Index
    df = df.reset_index()
    # Drop old index column
    df = df.drop(columns="index")
    return df

### Exercise One, ARP Poisoning
$\color{red}{\text{Note:}}$ Don't forget to run **Utility Functions and Configs** befor executing below cell.

In [5]:
pcap = sniff(iface="s1-eth2", filter="icmp",count=12)

df = pretty(pcap)

# Return Src Address, Dst Address, Src Port, Dst Port, IP Flags, TCP Flags and MSS
df[['src','dst','type']]

,src,src,dst,dst,type,type
0,00:00:00:00:00:03,10.0.0.3,00:00:00:00:00:02,10.0.0.1,IP,Echo Request
1,00:00:00:00:00:02,10.0.0.3,00:00:00:00:00:01,10.0.0.1,IP,Echo Request
2,00:00:00:00:00:01,10.0.0.1,00:00:00:00:00:02,10.0.0.3,IP,Echo Reply
3,00:00:00:00:00:02,10.0.0.1,00:00:00:00:00:03,10.0.0.3,IP,Echo Reply
4,00:00:00:00:00:03,10.0.0.3,00:00:00:00:00:02,10.0.0.1,IP,Echo Request
5,00:00:00:00:00:02,10.0.0.3,00:00:00:00:00:01,10.0.0.1,IP,Echo Request
6,00:00:00:00:00:01,10.0.0.1,00:00:00:00:00:02,10.0.0.3,IP,Echo Reply
7,00:00:00:00:00:02,10.0.0.1,00:00:00:00:00:03,10.0.0.3,IP,Echo Reply
8,00:00:00:00:00:03,10.0.0.3,00:00:00:00:00:02,10.0.0.1,IP,Echo Request
9,00:00:00:00:00:02,10.0.0.3,00:00:00:00:00:01,10.0.0.1,IP,Echo Request
